# Day 2 notebook

The objectives of this notebook are to practice
* the biological concepts of 
  * the translation of RNA to protein
  * the consequences of mutations in protein-coding DNA
  * gene regulatory networks
* the Python concepts of 
  * creating and using Python modules
  * reading input from files
  * calling functions on lists of arguments using unpacking

## Translation of RNA to protein

### PROBLEM 1 - RNA fragment translation (1 POINT)

Continuing our work on writing functions that work with DNA, RNA, and protein sequences, let us write a function that translates all codons of a string representing an RNA fragment, where we again assume that the first three bases of the RNA correspond to a codon and that the length of RNA is a multiple of three. At this point, we will not require that the first codon is the start codon and that the last codon is a stop codon.

You will likely want to copy some of your functions from the Day 1 notebook here to help with implementing this function.  If you wish, you may create a Python module with those functions (see the related Problem 8 below).  Also, as in the previous in-class activities, you will likely find the `join` method of the empty string convenient for concatenating all of the strings in a list together.  For example,

In [ ]:
bases = ["A", "C", "G", "T"]
''.join(bases)

In [ ]:
### BEGIN SOLUTION template=""
import translation
### END SOLUTION

def translate_rna_fragment(rna):
### BEGIN SOLUTION
    amino_acids = [translation.translate_codon(codon) 
                   for codon in translation.codons(rna)]
    return ''.join(amino_acids)
### END SOLUTION

In [ ]:
# tests for translate_rna_fragment
assert translate_rna_fragment("UUUGCGACUUAU") == "FAUY", "Failed on input 'UUUGCGACUUAU'"
assert translate_rna_fragment("ACG") == "U", "Failed on input 'UGA'"
assert translate_rna_fragment("") == "", "Failed on the empty string"
print("SUCCESS: translate_rna_fragment passed all tests!")

### PROBLEM 2 - RNA translation (1 POINT)

In this problem we step up the complexity and perform RNA translation of an entire RNA molecule.  Recall that RNA translation of a full RNA molecules involves:
1. Scanning for the first occurrence of a start codon ("AUG")
2. Translation of the start codon and successive codons in the same reading frame
3. Termination of translation when a stop codon is reached

You may assume that the RNA molecule contains a start codon as well as a downstream, in-frame, stop codon.

In [ ]:
def translate_rna(rna):
### BEGIN SOLUTION
    # Find the first start codon
    start_codon = "AUG"
    start_codon_position = rna.find(start_codon)
    codons_from_start = translation.codons(rna[start_codon_position:])
    amino_acids = []
    for codon in codons_from_start:
        amino_acid = translation.translate_codon(codon)
        if amino_acid == "*":
            break
        amino_acids.append(amino_acid)
    return ''.join(amino_acids)
### END SOLUTION

In [ ]:
# tests for translate_rna
assert translate_rna("AUGACUUAUUAG") == "MUY", "Failed on input 'AUGACUUAUUAG'"
assert translate_rna("AAUGACUUAUUAG") == "MUY", "Failed on input 'AAUGACUUAUUAG'"
assert translate_rna("AUGACUUAUUAGCT") == "MUY", "Failed on input 'AUGACUUAUUAGCT'"
assert translate_rna("AAUGACUUAUUAGCT") == "MUY", "Failed on input 'AAUGACUUAUUAGCT'"
print("SUCCESS: translate_rna passed all tests!")

### PROBLEM 3 - Consequences of the deltaF508 mutation in CFTR (2 POINTS)

We will now revisit the CFTR gene fragment that we saw in the day 1 notebook and examine how the deltaF508 mutation impacts the resulting amino acid sequence of the encoded protein.  Below is again the sequence of the CFTR gene fragment.

In [ ]:
cftr_gene_fragment = ("ACTTCACTTCTAATGGTGATTATGGGAGAACTGGAGCCTTCAGAGGGTAA"
                      "AATTAAGCACAGTGGAAGAATTTCATTCTGTTCTCAGTTTTCCTGGATTA"
                      "TGCCTGGCACCATTAAAGAAAATATCATCTTTGGTGTTTCCTATGATGAA"
                      "TATAGATACAGAAGCGTCATCAAAGCATGCCAACTAGAAGAG")
print(cftr_gene_fragment)

And here is the sequence of the mutated CFTR gene fragment, which has bases 129, 130, and 131 (1-based coordinates) removed.

In [ ]:
deltaf508_fragment = cftr_gene_fragment[:128] + cftr_gene_fragment[131:]
print(deltaf508_fragment)

Recall that the removed bases were

In [ ]:
print(cftr_gene_fragment[128:131])

Using your translation function and considering the coordinates of the deltaF508 mutation, determine
1. how many codons of the non-mutant fragment overlap with this deletion
2. which single amino acid is effectively deleted from the resulting mutant RNA fragment

Note that this CFTR sequence is only a fragment (substring) of the entire CFTR gene and so will not begin with a start codon or end with a stop codon.  You will also need to know that the first three bases of the fragment are a codon.

Answer the above two questions by assigning your answers to the variables in the code below.

In [ ]:
# Assign to the variable 'num_codons_overlapping_with_deletion' an integer
### BEGIN SOLUTION template=num_codons_overlapping_with_deletion = ?
num_codons_overlapping_with_deletion = 2
### END SOLUTION
# Assign to the variable 'amino_acid_deleted' a single character string
### BEGIN SOLUTION template=amino_acid_deleted = ?
amino_acid_deleted = "F"
### END SOLUTION

In [ ]:
# test for num_codon_overlapping_with_deletion
assert isinstance(num_codons_overlapping_with_deletion, int), "num_codons_overlapping_with_deletion is not an integer"
### BEGIN HIDDEN TESTS
assert num_codons_overlapping_with_deletion == 2, "num_codons_overlapping_with_deletion is incorrect"
print("SUCCESS: num_codons_overlapping_with_deletion passed all tests!")
### END HIDDEN TESTS

In [ ]:
# test for amino_acid_deleted
assert isinstance(amino_acid_deleted, str), "amino_acid_deleted is not a string"
### BEGIN HIDDEN TESTS
assert amino_acid_deleted == "F", "amino_acid_deleted is incorrect"
print("SUCCESS: amino_acid_deleted passed all tests!")
### END HIDDEN TESTS

## The lac operon gene regulatory network
For the remainder of this notebook, we will practice with very simple modeling of the lac operon gene regulatory network that was described in lecture.  

We will add one more component to this network, which is the *CAP* protein.  In an environment with low levels of glucose the CAP protein binds to the regulatory region for the lac operon and increases transcription of the lac operon, but only when *lacI* is not bound to the regulatory region.  This allows the cell to try to make up for the lack of glucose by increasing its metabolism of lactose instead.

The entire network, along with the deterministic functions/circuits for each downstream variable, is defined by the figure below.  The `lactose`, `lacI`, `glucose`, and `CAP` input variables take the value `True` when they are present and the value `False` when they are absent.  The `lacI_bound` and `CAP_bound` variables are also boolean valued and depend on a subset of the input variables.  The `lacZ` variable represents the transcription level of the lac operon, and can take one of three values: "absent", "low", and "high."

![Lac circuit](network.png)

We will define this network in Python using a series of very simple functions, which can then be composed.

### PROBLEM 4 - lacI_bound (1 POINT)
Write a function that specifies the lacI-bound circuit, as defined in the figure above.  This should be a simple one-liner that uses a boolean expression.

In [ ]:
def lacI_bound(lactose, lacI):
### BEGIN SOLUTION
    return not lactose and lacI
### END SOLUTION

In [ ]:
# tests for function lacI_bound
assert lacI_bound(False, False) == False, "Failed on input False, False"
assert lacI_bound(False, True) == True, "Failed on input False, True"
assert lacI_bound(True, False) == False, "Failed on input True, False"
assert lacI_bound(True, True) == False, "Failed on input True, True"
print("SUCCESS: lacI_bound passed all tests!")

### PROBLEM 5 - CAP_bound (1 POINT)
Write a function that specifies the CAP-bound circuit, as defined in the figure above.  This should also be a simple one-liner that uses a boolean expression.

In [ ]:
def CAP_bound(glucose, CAP):
### BEGIN SOLUTION
    return not glucose and CAP
### END SOLUTION

In [ ]:
# tests for function CAP_bound
assert CAP_bound(False, False) == False, "Failed on input False, False"
assert CAP_bound(False, True) == True, "Failed on input False, True"
assert CAP_bound(True, False) == False, "Failed on input True, False"
assert CAP_bound(True, True) == False, "Failed on input True, True"
print("SUCCESS: CAP_bound passed all tests!")

### PROBLEM 6 - lacZ (1 POINT)
Write a function that specifies the lacZ circuit, as defined in the figure above.

In [ ]:
def lacZ(is_lacI_bound, is_CAP_bound):
### BEGIN SOLUTION
    if is_lacI_bound:
        return "absent"
    elif is_CAP_bound:
        return "high"
    else:
        return "low"
### END SOLUTION

In [ ]:
# tests for function lacZ
assert lacZ(False, False) == "low", "Failed on input False, False"
assert lacZ(False, True) == "high", "Failed on input False, True"
assert lacZ(True, False) == "absent", "Failed on input True, False"
assert lacZ(True, True) == "absent", "Failed on input True, True"
print("SUCCESS: lacZ passed all tests!")

### PROBLEM 7 - lacZ full circuit (1 POINT)
Finally, let us write a function that gives the value of lacZ depending on the input variables `lactose`, `lacI`, `glucose`, and `CAP`.  Use composition of the functions above to accomplish this in a simple one-liner.

In [ ]:
def lacZ_full_circuit(lactose, lacI, glucose, CAP):
### BEGIN SOLUTION
    return lacZ(lacI_bound(lactose, lacI), CAP_bound(glucose, CAP))
### END SOLUTION

In [ ]:
# tests for function lacZ_full_circuit
# note that these are only a subset of the possible inputs
assert lacZ_full_circuit(False, False, False, False) == "low", "Failed on input False, False, False, False"
assert lacZ_full_circuit(False, False, False, True) == "high", "Failed on input False, False, False, True"
assert lacZ_full_circuit(False, True, False, True) == "absent", "Failed on input False, True, False, True"
assert lacZ_full_circuit(False, True, True, True) == "absent", "Failed on input False, True, True, True"
print("SUCCESS: lacZ_full_circuit passed all tests!")

### PROBLEM 8 - creating and using a module (1 POINT)
We will now practice with creating a Python module.  Modules make it easy to reuse your code and keep related functions, variables, and classes in a separate namespace.

For this problem, you are to do the following:
1. Create a new file `lac_network.py` in your workspace by clicking on the jupyter icon above, clicking on the "new" drop down menu at the right of the interface, and selecting "text file".
2. Copy the Python code for each of your lac circuit functions that you have written above into the `lac_network.py` file.
3. Import your `lac_network` module within this notebook and test that you can call functions defined within the module.

*Important note regarding module import in notebooks*: If you have already run the import statement for a module in a notebook and then make a change to the source code of the module, using the `import` statement again will not use the updated code.  You can either restart the Python kernel and re-run the cell with the `import` statement (recommended) or use the `reload` function from the [`importlib`](https://docs.python.org/3.6/library/importlib.html) standard module. 

In [ ]:
# tests for the lac_network module
import lac_network
assert lac_network.lacZ_full_circuit(False, False, False, False) == "low", "lac_network.lacZ_full_circuit failed"
assert lac_network.lacI_bound(False, False) == False, "lac_network.lacI_bound failed"
assert lac_network.CAP_bound(False, False) == False, "lac_network.CAP_bound failed"
assert lac_network.lacZ(False, False) == "low", "lac_network.lacZ failed"
print("SUCCESS: lacZ_network module passed all tests!")

### PROBLEM 9 - reading from a file (1 POINT)
We will now practice with reading data from a text file.  In particular, we will read a file that contains some input values for the `lacZ_full_circuit`, with a different set of input values on each line.  The input values will be specified for the variables `lactose`, `lacI`, `glucose`, and `CAP`, and in that order.  On each line, the values will be separated by one or more whitespace characters.

Write a function that takes as input the name of a file containing input values for the `lacZ_full_circuit` function and outputs a list of lists of the input values.  The first tuple in the list should correspond to the input values from the first line of the file, and so on for the other lines in the file.

You will likely find the following function, which converts the strings "True" and "False" to Python boolean values, of use.

You will also likely find the `split` method of string functions helpful for converting each line of the file into a list of tokens.

In [ ]:
def str_to_bool(s):
    if s == 'True':
         return True
    elif s == 'False':
         return False
    else:
         raise ValueError("Input is neither 'True' nor 'False': " + s)

In [ ]:
def read_lacZ_full_circuit_inputs(filename):
    ### BEGIN SOLUTION
    return [[str_to_bool(token) for token in line.split()] for line in open(filename)]
    ### END SOLUTION

In [ ]:
# tests for read_lacZ_full_circuit_inputs
assert read_lacZ_full_circuit_inputs("lac_network_inputs.txt") == [[False, False, False, False],
                                                                   [False, False, False, True],
                                                                   [False, False, True, False],
                                                                   [False, False, True, True]]
print("SUCCESS: read_lacZ_full_circuit_inputs passed all tests!")

### PROBLEM 10 - calling functions with arguments in a list (1 POINT)
Finally, we will practice calling a function that takes multiple arguments when the arguments are packed within a single list or tuple.  For this we will use Python's [unpacking](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists) syntax.

For example, suppose we have a number of words in a list, and we want to use the print statement to print all of the words on a single line, separated by a single space character.  We can use unpacking to make this easy.

In [ ]:
some_words = ["Here", "are", "some", "words"]

In [ ]:
print(some_words)

In [ ]:
print(*some_words)

Notice the difference in the output from the two print statements above.  In the first statement, `print` is given a single argument, a list of strings.  In the second statement, `print` is given multiple arguments, each an element from the list `some_words`.

Use this argument unpacking technique to write a function that calls `lacZ_full_circuit` on each of a list of input value lists, such as is returned by the `read_lacZ_full_circuit_inputs` function you defined in the previous problem.  The function should return a list of the corresponding outputs from the `lacZ_full_circuit` function.

In [ ]:
def call_lacZ_full_circuit_on_list(inputs_list):
    ### BEGIN SOLUTION
    return [lacZ_full_circuit(*inputs) for inputs in inputs_list]
    ### END SOLUTION

In [ ]:
# tests for call_lacZ_full_circuit_on_list
test1_list = [[False, False, False, False],
              [False, False, False, True],
              [False, False, True, False],
              [False, False, True, True]]
assert call_lacZ_full_circuit_on_list(test1_list) == ['low', 'high', 'low', 'low'], "Failed on test1_list"
test2_list = [[False, False, False, True]]
assert call_lacZ_full_circuit_on_list(test2_list) == ['high'], "Failed on test2_list"
print("SUCCESS: call_lacZ_full_circuit_on_list passed all tests!")

## Submitting your notebook

Congratulations, you have reached the end of this notebook!  To double check that you have completed all of the problems correctly, you are encouraged to restart Python and run all of the cells in the notebook from beginning to end.  You can do so automatically by running the command "Restart & Run All" from the "Kernel" menu.  After running this command, look through your noteobok and check that all of the tests ran successfully.

To submit your work, press the big blue "Submit" button at the top of this web page.  You may submit as many times as you wish and your final grade will be based on your most recent submission.  After you submit, a grade report will become available telling you how many points you received on each problem in the notebook.